In [3]:
import glob
import numpy as np
import pickle
import torch
import importlib
import webdataset as wds
import random
import h5py
import matplotlib.pyplot as plt 
import wandb

# Configurations

In [ ]:
importlib.reload(plot)
plot.plot(results_folder="./train_logs/", 
     exp_name="pretrainNOV16_1700670984.431539*", 
                    num_runs_analyze=10000)

In [ ]:
importlib.reload(plot)
plot.plot_learning_curve(results_folder="./train_logs/", 
     exp_name="finetune_subj1_perf_vs_numsession_NOV24*", 
                    num_runs_analyze=10)

# Pretraining 

In [ ]:
importlib.reload(plot)
plot.plot_pretraining(results_folder="./train_logs/", 
     exp_name="pretrainNOV16_1700670984*", 
                    num_runs_analyze=100000)

# Wandb

In [ ]:
import wandb
import seaborn as sns
api = wandb.Api()
ENTITY = "anhhuyalex"
PROJECT = "eye_int"
# Filter runs by group name
runs = api.runs(f"{ENTITY}/{PROJECT}")#, filters={"group": "finetune_numsessions"})

# Download specific data from each run
num_sessions = []
test_bwd_pct_correct = []
test_fwd_pct_correct = []
for run in runs:
    if run.state == "finished" and run.config.get("num_sessions", None) is not None:
        num_sessions.append(   run.config["num_sessions"])
        test_bwd_pct_correct.append( run.summary["test/test_bwd_pct_correct"])
        test_fwd_pct_correct.append( run.summary["test/test_fwd_pct_correct"])
sns.set_theme(style="whitegrid")
ax = sns.lineplot( x=num_sessions, y=test_fwd_pct_correct, marker="o", label="test_fwd_pct_correct")
ax = sns.lineplot( x=num_sessions, y=test_bwd_pct_correct, marker="o", label="test_bwd_pct_correct")

ax.set_title("test_fwd_pct_correct")
ax.legend(loc="lower right")
ax.set_xlabel("num_sessions")

In [ ]:
import wandb
import seaborn as sns
api = wandb.Api()
ENTITY = "anhhuyalex"
PROJECT = "eye_int"
# Filter runs by group name
runs = api.runs(f"{ENTITY}/{PROJECT}", filters={"group": "finetune_numsamples"})

# get samples pool
filter_samples = "./cache/subj01_permuted_samples.txt"
with open(filter_samples) as f:
    samples_pool = f.read().splitlines()
 
# Download specific data from each run
num_samples = []
left_out = []
test_bwd_pct_correct = []
test_fwd_pct_correct = []
for run in runs:
    if run.state == "finished" and run.config.get("num_samples", None) == 6999:
        samples_list = run.config["samples_list"]
        left_out.append([x for x in samples_pool[:7000] if x not in samples_list])
        test_bwd_pct_correct.append( run.history().iloc[19]["test/test_bwd_pct_correct"])
        test_fwd_pct_correct.append( run.history().iloc[19]["test/test_fwd_pct_correct"])
        
sns.set_theme(style="whitegrid")
ax = sns.histplot(  x=test_fwd_pct_correct,   label="test_fwd_pct_correct")
ax = sns.histplot( x=test_bwd_pct_correct,  label="test_bwd_pct_correct")


ax.legend(loc="lower right")


In [13]:
import wandb
import seaborn as sns
api = wandb.Api()
ENTITY = "anhhuyalex"
PROJECT = "eye_int"
# Filter runs by group name
runs = api.runs(f"{ENTITY}/{PROJECT}", filters={"group": "check_variance_of_run"})

# get samples pool
filter_samples = "./cache/subj01_permuted_samples.txt"
with open(filter_samples) as f:
    samples_pool = f.read().splitlines()
 
# Download specific data from each run
num_samples = []
left_out = []
test_bwd_pct_correct = []
test_fwd_pct_correct = []
for run in runs:
    # run.delete()
#     if run.state == "finished" and run.config.get("num_samples", None) == 6999:
    samples_list = run.config["samples_list"]
    left_out.append([x for x in samples_pool[:7000] if x not in samples_list])
    print("left out", left_out[-1])
    try:
        test_bwd_pct_correct.append( run.history().iloc[19]["test/test_bwd_pct_correct"])
        test_fwd_pct_correct.append( run.history().iloc[19]["test/test_fwd_pct_correct"])
    except:
        continue
sns.set_theme(style="whitegrid")
ax = sns.histplot(  x=test_fwd_pct_correct,   label="test_fwd_pct_correct")
ax = sns.histplot( x=test_bwd_pct_correct,  label="test_bwd_pct_correct")


ax.legend(loc="lower right")


left out ['sample000003988']
left out ['sample000003988']
left out ['sample000003988']
left out ['sample000003988']
left out ['sample000003988']
left out ['sample000003988']
left out ['sample000003988']
left out ['sample000003988']
left out ['sample000003988']
left out ['sample000003988']
left out ['sample000003988']
left out ['sample000003988']
left out ['sample000003988']
left out ['sample000003988']
left out ['sample000003988']
left out ['sample000003988']
left out ['sample000003988']
left out ['sample000003988']
left out ['sample000003988']
left out ['sample000003988']


KeyboardInterrupt: 

In [ ]:
worst_performing_prunes_bwd = [left_out[i] for i in np.argsort(test_bwd_pct_correct)[:25]]
best_performing_prunes_bwd = [left_out[i] for i in np.argsort(test_bwd_pct_correct)[-25:]]
print(worst_performing_prunes_bwd)
print(best_performing_prunes_bwd)
print([i for i in worst_performing_prunes_bwd if i in best_performing_prunes_bwd])

worst_performing_prunes_fwd = [left_out[i] for i in np.argsort(test_fwd_pct_correct)[:25]]
best_performing_prunes_fwd = [left_out[i] for i in np.argsort(test_fwd_pct_correct)[-25:]]
print(worst_performing_prunes_fwd)
print(best_performing_prunes_fwd)
print([i for i in best_performing_prunes_fwd if i in worst_performing_prunes_fwd])

In [ ]:
data_path="/fsx/proj-fmri/shared/mindeyev2_dataset"
train_url = f"{data_path}/wds/subj01/train/" + "{0.." + f"39" + "}.tar"
print(f"subj01 with {train_url} train_url")
def my_split_by_node(urls): return urls
def filter_samples_func(sample):
    if sample["__key__"] not in worst_performing_prunes:
        return None
    return sample
train_data = wds.WebDataset(train_url,resampled=True,nodesplitter=my_split_by_node)\
                    .decode("torch") \
                    .rename(behav="behav.npy", past_behav="past_behav.npy", future_behav="future_behav.npy", olds_behav="olds_behav.npy")\
                    .to_tuple(*["__key__","behav", "past_behav", "future_behav", "olds_behav"])
train_dl = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=False, drop_last=True, pin_memory=True)
data_type = torch.float16
f = h5py.File(f'{data_path}/coco_images_224_float16.hdf5', 'r')
images = f['images'][:]
images = torch.Tensor(images).to("cpu").to(data_type)

In [ ]:
# BACKWARD PERFORMANCE IS WORST IF THESE IMAGES OMITTED
sns.set_style("white")
seen_images = set()
fig, ax = plt.subplots(5, 5, figsize=(15, 15))
worst_images = {}
for iter, (key, behav0, past_behav0, future_behav0, old_behav0) in enumerate(train_dl):  
    if key[0] in seen_images: continue
    try:
        rank = worst_performing_prunes_bwd.index(key)
    except:
        continue
    
    seen_images.add(key[0])
    worst_images[rank] = images[behav0[:,0,0].cpu().long()].float()
    if len(seen_images) == 25: break
for i in range(25):
    ax[i%5,i//5].imshow(worst_images[i][0].permute(1,2,0))
plt.show()

In [ ]:
# BACKWARD PERFORMANCE IS WORST IF THESE IMAGES OMITTED
sns.set_style("white")
seen_images = set()
fig, ax = plt.subplots(5, 5, figsize=(15, 15))
worst_images = {}
for iter, (key, behav0, past_behav0, future_behav0, old_behav0) in enumerate(train_dl):  
    if key[0] in seen_images: continue
    try:
        rank = worst_performing_prunes_fwd.index(key)
    except:
        continue
    
    seen_images.add(key[0])
    worst_images[rank] = images[behav0[:,0,0].cpu().long()].float()
    if len(seen_images) == 25: break
for i in range(25):
    ax[i%5,i//5].imshow(worst_images[i][0].permute(1,2,0))
plt.show()

In [ ]:
# BACKWARD PERFORMANCE IS WORST IF THESE IMAGES OMITTED
sns.set_style("white")
seen_images = set()
fig, ax = plt.subplots(5, 5, figsize=(15, 15))
best_images = {}
for iter, (key, behav0, past_behav0, future_behav0, old_behav0) in enumerate(train_dl):  
    if key[0] in seen_images: continue
    try:
        rank = best_performing_prunes_bwd.index(key)
    except:
        continue
    
    seen_images.add(key[0])
    best_images[rank] = images[behav0[:,0,0].cpu().long()].float()
    if len(seen_images) == 25: break
for i in range(25):
    ax[i%5,i//5].imshow(best_images[i][0].permute(1,2,0))
plt.show()

In [ ]:
# BACKWARD PERFORMANCE IS WORST IF THESE IMAGES OMITTED
sns.set_style("white")
seen_images = set()
fig, ax = plt.subplots(5, 5, figsize=(15, 15))
best_images = {}
for iter, (key, behav0, past_behav0, future_behav0, old_behav0) in enumerate(train_dl):  
    if key[0] in seen_images: continue
    try:
        rank = best_performing_prunes_fwd.index(key)
    except:
        continue
    
    seen_images.add(key[0])
    best_images[rank] = images[behav0[:,0,0].cpu().long()].float()
    if len(seen_images) == 25: break
for i in range(25):
    ax[i%5,i//5].imshow(best_images[i][0].permute(1,2,0))
plt.show()

In [ ]:
from functools import partial
from diffusers.models.vae import Decoder
class BrainNetwork(nn.Module):
    def __init__(self, out_dim=768, in_dim=15724, clip_size=768, h=4096, n_blocks=4, norm_type='ln', act_first=False, drop=.15, blurry_dim=16):
        super().__init__()
        self.blurry_dim = blurry_dim
        norm_func = partial(nn.BatchNorm1d, num_features=h) if norm_type == 'bn' else partial(nn.LayerNorm, normalized_shape=h)
        act_fn = partial(nn.ReLU, inplace=True) if norm_type == 'bn' else nn.GELU
        act_and_norm = (act_fn, norm_func) if act_first else (norm_func, act_fn)
        self.lin0 = nn.Linear(in_dim, h)
        self.mlp = nn.ModuleList([
            nn.Sequential(
                nn.Linear(h, h),
                *[item() for item in act_and_norm],
                nn.Dropout(drop)
            ) for _ in range(n_blocks)
        ])
        self.lin1 = nn.Linear(h, out_dim, bias=True)
        self.blin1 = nn.Linear(out_dim, blurry_dim, bias=True)
        self.n_blocks = n_blocks
        self.clip_size = clip_size
        self.clip_proj = nn.Sequential(
            nn.LayerNorm(clip_size),
            nn.GELU(),
            nn.Linear(clip_size, 2048),
            nn.LayerNorm(2048),
            nn.GELU(),
            nn.Linear(2048, 2048),
            nn.LayerNorm(2048),
            nn.GELU(),
            nn.Linear(2048, clip_size)
        )
        self.upsampler = Decoder(
                in_channels=64,
                out_channels=4,
                up_block_types=["UpDecoderBlock2D","UpDecoderBlock2D","UpDecoderBlock2D"],
                block_out_channels=[64, 128, 256],
                layers_per_block=1,
            )
        
    def forward(self, x):
        x = self.lin0(x)
        residual = x
        for res_block in range(self.n_blocks):
            x = self.mlp[res_block](x)
            x += residual
            residual = x
        x = x.reshape(len(x), -1)
        x = self.lin1(x)
        b = self.blin1(x)
        b = self.upsampler(b.reshape(len(b), -1, 7, 7))
        c = self.clip_proj(x.reshape(len(x), -1, self.clip_size))
        return c, b

model.backbone = BrainNetwork(h=2048, in_dim=hidden_dim, clip_size=clip_emb_dim, out_dim=clip_emb_dim*clip_seq_dim, blurry_dim=64*7*7) 
utils.count_params(model.backbone)
utils.count_params(model)

b = torch.randn((2,hidden_dim))
print(b.shape)
clip_, blur_ = model.backbone(b)
print(clip_.shape, blur_.shape)

In [ ]:
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
opt_grouped_parameters = [
    {'params': [p for n, p in model.ridge.named_parameters()], 'weight_decay': 1e-2},
    {'params': [p for n, p in model.backbone.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 1e-2},
    {'params': [p for n, p in model.backbone.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
]

optimizer = torch.optim.AdamW(opt_grouped_parameters, lr=max_lr, betas=(0.9, 0.95))

if lr_scheduler_type == 'linear':
    lr_scheduler = torch.optim.lr_scheduler.LinearLR(
        optimizer,
        total_iters=int(num_epochs*(num_train*num_devices//batch_size)),
        last_epoch=-1
    )
elif lr_scheduler_type == 'cycle':
    total_steps=int(num_epochs*(num_train*num_devices//batch_size))
    lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer, 
        max_lr=max_lr,
        total_steps=total_steps,
        final_div_factor=1000,
        last_epoch=-1, pct_start=2/num_epochs
    )
    
def save_ckpt(tag):    
    ckpt_path = outdir+f'/{tag}.pth'
    print(f'saving {ckpt_path}',flush=True)
    unwrapped_model = accelerator.unwrap_model(model)
    try:
        torch.save({
            'epoch': epoch,
            'model_state_dict': unwrapped_model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'lr_scheduler': lr_scheduler.state_dict(),
            'train_losses': losses,
            'test_losses': test_losses,
            'lrs': lrs,
            }, ckpt_path)
    except:
        print("Couldn't save... moving on to prevent crashing.")
    del unwrapped_model
        
print("\nDone with model preparations!")
utils.count_params(model)

# Active learning prune by 100

In [1]:
import webdataset as wds 
import random
import torch
def my_split_by_node(urls): return urls

voxels = {}
for si, s in enumerate([1]):
    train_url = f"/weka/proj-fmri/shared/mindeyev2_dataset/wds/subj0{s}/train/" + "{0.." + f"39" + "}.tar"
    print(f"subj01 training with {train_url} train_url")
    # Load hdf5 data for betas, but don't put everything into memory
    f = h5py.File(f'/weka/proj-fmri/shared/mindeyev2_dataset/betas_all_subj0{s}_fp32.hdf5', 'r')
    betas = f['betas'][:]
    del f
train_url = "/weka/proj-fmri/shared/mindeyev2_dataset/wds/subj01/new_train/{0..39}.tar"
train_data = wds.WebDataset(train_url,resampled=True,nodesplitter=my_split_by_node)\
                        .shuffle(750, initial=2500, rng=random.Random(1))\
                        .decode("torch")\
                       .rename(behav="behav.npy", past_behav="past_behav.npy", future_behav="future_behav.npy", olds_behav="olds_behav.npy")\
                       .to_tuple(*["__key__", "behav", "past_behav", "future_behav", "olds_behav"]) 
train_dl = torch.utils.data.DataLoader(train_data, batch_size=512, shuffle=False, drop_last=True, pin_memory=True) 
keys = set() 
# for iter, (key) in enumerate(train_dl):    
for iter, (key, behav0, past_behav0, future_behav0, old_behav0) in enumerate(train_dl): 
    
#     print (behav0[:,0,0].cpu().long())
#     print (past_behav0[:,0,0].cpu().long())
    [keys.add( i.item() ) for i in behav0[:,0,0].cpu().long()]
#     print ((keys))
    if iter % 10 == 1: print (len(keys))
    if iter > 1000: break

830
3942
5985
6976
7593
7937
8316
8518
8668
8866
8899
8931
8931
8931
8964
8964
8964
8964
8964
9000
9000


KeyboardInterrupt: 

In [6]:

import copy
import numpy as np
keys = np.load("./cache/training_set.npy")
np.random.shuffle(keys) 
print (len(keys), keys[:20])
num_keys_chosen = 100
for seed in range(len(keys)//num_keys_chosen):
    print ("len", len(keys[seed * num_keys_chosen: (seed+1) * num_keys_chosen]))
    np.save (f"./cache/keys_{num_keys_chosen}_from_0_seed_{seed}.npy", keys[seed * num_keys_chosen: (seed+1) * num_keys_chosen])
    

8000 [41540  3184 39692  7591 65417 45166 53785 63090 62372 58812 67111 25527
 34318 70615 45635 10631 51451 48697 61180 61875]
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100


In [2]:
import numpy as np
t = np.load("./cache/training_set.npy")
v = np.load("./cache/validation_set.npy")
assert not any(_t in set(v) for _t in t), "Found an element of t in v" 

t, v

(array([42121, 41062, 14842, ..., 33656, 15692, 36417]),
 array([50203, 30596, 72074, 49510, 67375,  1886, 52069, 26515, 66383,
         6157, 12231, 25447,  3983, 12264, 35996, 25233, 10825, 67167,
        57733,   653, 13176, 10234, 11628, 52509, 10209, 42193,  8707,
        47738, 33972, 55986, 29512,  3177, 52515, 72527, 26816, 54853,
        54255, 24635, 10410, 42205, 14688,  3355, 11305, 18797, 44809,
        38718, 33376, 72911, 11455, 40795,  2709, 31749, 54415,  8227,
        39315, 41976, 11992, 58645, 67276, 36134, 21969, 60419, 49616,
        32630, 16735,  7260, 32336, 19818,  9562, 66374, 26693, 65703,
        45035, 72710, 37846, 67862, 26659, 70218, 51099, 53066, 49934,
        47918, 61065,  6138, 34558, 44150, 65629,  7539, 53193, 39920,
        56972, 36996, 28118, 37542, 61804, 25032, 34826, 22790,   884,
         6412, 32087, 72373,  9384, 61417, 17373, 20477, 43501, 19425,
        18156, 17843,  2307, 72002, 53819, 50259, 67762, 10674, 32988,
         1233, 53163

In [2]:
import wandb
import seaborn as sns
import numpy as np
from collections import defaultdict
api = wandb.Api()

ENTITY = "anhhuyalex"
PROJECT = "eye_int"
# Filter runs by group name
# runs = api.runs(f"{ENTITY}/{PROJECT}", filters={"group": "jan22_2221_pruning_500"})
runs = api.runs(f"{ENTITY}/{PROJECT}", filters={"group": "jan22_2221_pruning_6000_from_3500"})
 
 
# Download specific data from each run
num_samples = []
left_out = []
test_bwd_pct_correct = defaultdict(list)
test_fwd_pct_correct = defaultdict(list)
test_both_pct_correct = defaultdict(list)
num_runs = 0
for run in runs:
    # run.delete()
#     if run.state == "finished" and run.config.get("num_samples", None) == 6999:
    
    try:
        print (run.summary["_step"])
        if (run.summary["_step"] != 14): continue
    except:
        continue 
    # print ("run.config", run.config)
    offline_logfile = run.config["offline_logfile"] 
    # if  [int(i) for i in offline_logfile[1:100].split(" ") if i != ""] != [70420, 2598, 63069, 32343, 37942, 25934, 32988, 15835, 50918, 48882, 28161, 13467, 6373, 1247, 41268, 66508, 31]: continue
    # num_runs += 1
   

    print ("offline_logfile",offline_logfile)
    
    # display (run.history())#["filter_samples"].iloc[0])
    max_bwd = run.history()["test/test_bwd_pct_correct"].max()
    max_fwd = run.history()["test/test_fwd_pct_correct"].max()
    test_bwd_pct_correct[offline_logfile].append(max_bwd)
    test_fwd_pct_correct[offline_logfile].append(max_fwd)
    test_both_pct_correct[offline_logfile].append((max_bwd+max_fwd)/2)
    
  
    try:
        test_bwd_pct_correct.append( run.history().iloc[19]["test/test_bwd_pct_correct"])
        test_fwd_pct_correct.append( run.history().iloc[19]["test/test_fwd_pct_correct"])
    except:
        continue
mean_test_bwd_pct_correct = {k: np.mean(v) for k,v in test_bwd_pct_correct.items()} 
mean_test_fwd_pct_correct = {k: np.mean(v) for k,v in test_fwd_pct_correct.items()}
mean_test_both_pct_correct = {k: np.mean(v) for k,v in test_both_pct_correct.items()}
best_test_bwd_pct_correct = sorted(mean_test_bwd_pct_correct.items(), key=lambda x: x[1], reverse=True) 
best_test_fwd_pct_correct = sorted(mean_test_fwd_pct_correct.items(), key=lambda x: x[1], reverse=True)
best_test_both_pct_correct = sorted(mean_test_both_pct_correct.items(), key=lambda x: x[1], reverse=True)
print ("best_test_fwd_pct_correct", best_test_fwd_pct_correct) 
print ("best_test_bwd_pct_correct", best_test_bwd_pct_correct)
print ("best_test_both_pct_correct", best_test_both_pct_correct)
# sns.set_theme(style="whitegrid")
# ax = sns.histplot(  x=test_fwd_pct_correct,   label="test_fwd_pct_correct")
# ax = sns.histplot( x=test_bwd_pct_correct,  label="test_bwd_pct_correct")


# ax.legend(loc="lower right")


14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_71.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_76.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_86.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_51.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_7.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_49.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_83.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_75.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_70.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_73.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_

14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_86.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_51.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_83.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_70.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_75.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_49.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_79.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_7.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_73.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_81.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_

14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_73.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_70.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_72.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_82.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_51.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_91.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_79.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_77.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_83.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_81.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000

14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_79.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_81.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_87.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_82.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_72.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_70.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_85.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_66.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_83.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_77.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000

14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_83.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_84.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_81.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_80.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_78.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_79.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_77.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_76.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_75.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_72.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000

In [5]:
import copy
import pickle 
print( [len(v) for k,v in test_bwd_pct_correct.items()] )
print([ len(v) for k,v in test_fwd_pct_correct.items()] )
print([  (v) for k,v in best_test_both_pct_correct ] )
best_filter_list, best_both = best_test_both_pct_correct[0]

print ("best_filter_list, best_both",best_filter_list, best_both, len(best_test_both_pct_correct))
 
with open(best_filter_list, "rb") as f:
    best_filter_samples = pickle.load(f)
best_filter_samples = best_filter_samples["filter_samples"]
print ("best_filter_samples", len(set(best_filter_samples)), best_filter_samples[:100])

other_keys = [k for k in keys if k not in best_filter_samples]
# verify that the keys are unique
print ("keys", len(keys), len(other_keys)) 
assert not any(x in set(other_keys) for x in best_filter_samples), "Found an element of best_filter_samples in other_keys"
num_keys_chosen = 500
for seed in range(100):
    np.random.shuffle(other_keys) 
    other_keys_500 = copy.deepcopy(other_keys[:num_keys_chosen]) 
    best_filter_samples_and_other_keys = np.concatenate([best_filter_samples, other_keys_500] )
    print(best_filter_samples_and_other_keys[5995:6005], best_filter_samples_and_other_keys[-10:], len(best_filter_samples_and_other_keys))
    np.save (f"./cache/keys_{len(set(best_filter_samples))+num_keys_chosen}_from_{9000-len(set(best_filter_samples))}_seed_{seed}.npy", best_filter_samples_and_other_keys)

[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
[0.8256666958332062, 0.8253333508968353, 0.821333360671997, 0.8210000216960907, 0.8196666896343231, 0.8190000176429748, 0.8186666905879975, 0.8183333396911621, 0.8180000185966492, 0.8180000126361847, 0.8170000195503235, 0.8166666746139526, 0.8163333594799042, 0.8160000264644622, 0.8146666824817658, 0.8143333494663239, 0.8143333435058594, 0.8143333435058594, 0.8140000104904175, 0.8140000104904175, 0.81366668343544, 0.81366668343544, 0.8136666774749756, 0.8136666715145111, 

[62047 18208 38834 42249 53899 58930 32541 69137 41202 55685] [31735  4750 24943 15685 53144 32531 31655 46943  7940 45548] 6500
[62047 18208 38834 42249 53899 43622   608 53396 35512  9296] [21048 25566  1743 28835 41536 43626 16367 45588 30311 24289] 6500
[62047 18208 38834 42249 53899 21107  4033  9408 43205 59565] [25916 34799 16259 55084 43457  2710 65803  8983 72606 64888] 6500
[62047 18208 38834 42249 53899 31724 21643 10054 19593 40996] [49510 51453 55248 15384 72002 35512 21447 52377  1233 31298] 6500
[62047 18208 38834 42249 53899 33319 53684 46613 14429 25029] [13102 66298  7316 47218 12085 53729 64647 33017 36432 65532] 6500
[62047 18208 38834 42249 53899 59006  7502 16743 40795 63283] [64195  7355 24216 40819 37083  2613 28280 65263  7919 56933] 6500
[62047 18208 38834 42249 53899 31982 11455 47121 15738 63739] [16310 58019  2735  8994 24635  5361 53729  1913  4470 46992] 6500
[62047 18208 38834 42249 53899 66776 48740 30213 46943 56750] [19260 44737  4439 66383 11490 3084

# Forward pruning

In [9]:
import wandb
import seaborn as sns
import numpy as np
from collections import defaultdict
api = wandb.Api()

ENTITY = "anhhuyalex"
PROJECT = "eye_int"
# Filter runs by group name
# runs = api.runs(f"{ENTITY}/{PROJECT}", filters={"group": "jan22_2221_pruning_500"})
runs = api.runs(f"{ENTITY}/{PROJECT}", filters={"group": "forwardprune_feb13_adding_100_from_0"})
 
 
# Download specific data from each run
num_samples = []
left_out = []
val_bwd_pct_correct = defaultdict(list)
val_fwd_pct_correct = defaultdict(list)
val_both_pct_correct = defaultdict(list)
num_runs = 0
for run in runs:
    # run.delete()
#     if run.state == "finished" and run.config.get("num_samples", None) == 6999:
    
    try:
        print (run.summary["_step"])
        if (run.summary["_step"] != 14): continue
    except:
        continue 
    # print ("run.config", run.config)
    offline_logfile = run.config["offline_logfile"] 
    # if  [int(i) for i in offline_logfile[1:100].split(" ") if i != ""] != [70420, 2598, 63069, 32343, 37942, 25934, 32988, 15835, 50918, 48882, 28161, 13467, 6373, 1247, 41268, 66508, 31]: continue
    # num_runs += 1
   

    print ("offline_logfile",offline_logfile)
    
    # display (run.history())#["filter_samples"].iloc[0])
    max_bwd = run.history()["val/val_bwd_pct_correct"].max()
    max_fwd = run.history()["val/val_fwd_pct_correct"].max()
    test_bwd_pct_correct[offline_logfile].append(max_bwd)
    test_fwd_pct_correct[offline_logfile].append(max_fwd)
    test_both_pct_correct[offline_logfile].append((max_bwd+max_fwd)/2)
    
  
    try:
        test_bwd_pct_correct.append( run.history().iloc[19]["test/test_bwd_pct_correct"])
        test_fwd_pct_correct.append( run.history().iloc[19]["test/test_fwd_pct_correct"])
    except:
        continue
mean_test_bwd_pct_correct = {k: np.mean(v) for k,v in test_bwd_pct_correct.items()} 
mean_test_fwd_pct_correct = {k: np.mean(v) for k,v in test_fwd_pct_correct.items()}
mean_test_both_pct_correct = {k: np.mean(v) for k,v in test_both_pct_correct.items()}
best_test_bwd_pct_correct = sorted(mean_test_bwd_pct_correct.items(), key=lambda x: x[1], reverse=True) 
best_test_fwd_pct_correct = sorted(mean_test_fwd_pct_correct.items(), key=lambda x: x[1], reverse=True)
best_test_both_pct_correct = sorted(mean_test_both_pct_correct.items(), key=lambda x: x[1], reverse=True)
print ("best_test_fwd_pct_correct", best_test_fwd_pct_correct) 
print ("best_test_bwd_pct_correct", best_test_bwd_pct_correct)
print ("best_test_both_pct_correct", best_test_both_pct_correct)
# sns.set_theme(style="whitegrid")
# ax = sns.histplot(  x=test_fwd_pct_correct,   label="test_fwd_pct_correct")
# ax = sns.histplot( x=test_bwd_pct_correct,  label="test_bwd_pct_correct")


# ax.legend(loc="lower right")


14
offline_logfile ./forward_pruning_cache/finetune_numsamplesforwardprune_keys_6000_from_6500_seed_99.log
14
offline_logfile ./forward_pruning_cache/finetune_numsamplesforwardprune_keys_6000_from_6500_seed_92.log
14
offline_logfile ./forward_pruning_cache/finetune_numsamplesforwardprune_keys_6000_from_6500_seed_33.log
14
offline_logfile ./forward_pruning_cache/finetune_numsamplesforwardprune_keys_6000_from_6500_seed_90.log
14
offline_logfile ./forward_pruning_cache/finetune_numsamplesforwardprune_keys_6000_from_6500_seed_87.log
14
offline_logfile ./forward_pruning_cache/finetune_numsamplesforwardprune_keys_6000_from_6500_seed_77.log
14
offline_logfile ./forward_pruning_cache/finetune_numsamplesforwardprune_keys_6000_from_6500_seed_84.log
14
offline_logfile ./forward_pruning_cache/finetune_numsamplesforwardprune_keys_6000_from_6500_seed_8.log
14
offline_logfile ./forward_pruning_cache/finetune_numsamplesforwardprune_keys_6000_from_6500_seed_86.log
14
offline_logfile ./forward_pruning_c

In [12]:
import copy
import pickle 
print( [len(v) for k,v in test_bwd_pct_correct.items()] )
print([ len(v) for k,v in test_fwd_pct_correct.items()] )
print([  (v) for k,v in best_test_both_pct_correct ] )
best_filter_list, best_both = best_test_both_pct_correct[0]

print ("best_filter_list, best_both",best_filter_list, best_both, len(best_test_both_pct_correct))
 
with open(best_filter_list, "rb") as f:
    best_filter_samples = pickle.load(f)
best_filter_samples = best_filter_samples["filter_samples"]
print ("best_filter_samples", len(set(best_filter_samples)), best_filter_samples[:100])

num_keys_chosen = 500
for seed in range(100):
    np.random.shuffle(best_filter_samples) 
    print(best_filter_samples[:(len(set(best_filter_samples)) - num_keys_chosen)] [-10:], len(best_filter_samples[:(len(set(best_filter_samples)) - num_keys_chosen)]))
    np.save (f"./cache/forwardprune_keys_{len(set(best_filter_samples))-num_keys_chosen}_from_{len(set(best_filter_samples))}_seed_{seed}.npy", best_filter_samples[:(len(set(best_filter_samples)) - num_keys_chosen)])

[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4]
[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4]
[0.8333333551883697, 0.8333333551883697, 0.8310000181198121, 0.8303333640098571, 0.8303333580493927, 0.8293333530426026, 0.8276666879653931, 0.8276666879653931, 0.8276666760444641, 0.8273333549499512, 0.8270000159740448, 0.8266666889190674, 0.8266666769981384, 0.8263333559036254, 0.8256666898727417, 0.8256666779518127, 0.8253333568572998, 0.824666690826416, 0.824666690826416, 0.8246666789054871, 0.824333351850509

# Plot results of active learning search

In [ ]:
import wandb
import seaborn as sns
import numpy as np
from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt
api = wandb.Api()

ENTITY = "anhhuyalex"
PROJECT = "eye_int"
# Filter runs by group name
# runs = api.runs(f"{ENTITY}/{PROJECT}", filters={"group": "jan22_2221_pruning_500"})
def load_runs(group_name, num_samples, run_type):
    runs = api.runs(f"{ENTITY}/{PROJECT}", filters={"group": group_name})
    
    
    # Download specific data from each run 
    test_bwd_pct_correct = []
    test_fwd_pct_correct = []
    test_both_pct_correct = []
    num_sample_list = []
    run_type_list = []
    for i, run in enumerate(runs):
     #     if run.state == "finished" and run.config.get("num_samples", None) == 6999:
        
        try:
            print (run.summary["_step"]) 
            
            if (run.summary["_step"] != 14): continue
#             if (len(run.config["samples_list"])== 41): continue
#             print (run.config["samples_list"], len(run.config["samples_list"]))
        except:
            continue 
        if group_name == "jan22_2221_pruning_1000":
            samples_list = run.config["samples_list"]
            print (samples_list[1:100])
            if [int(i) for i in samples_list[1:100].split(" ") if i != ""] != [70420, 2598, 63069, 32343, 37942, 25934, 32988, 15835, 50918, 48882, 28161, 13467, 6373, 1247, 41268, 66508, 31]: 
                run_type = "random"
            else:
                run_type = "backward"
                
            
        # print ("run.config", run.config)
        # offline_logfile = run.config["offline_logfile"] 
        # try: 
        #     print (len( run.config["samples_list"] ))
        # except:
        #     pass 
        # if  [int(i) for i in offline_logfile[1:100].split(" ") if i != ""] != [70420, 2598, 63069, 32343, 37942, 25934, 32988, 15835, 50918, 48882, 28161, 13467, 6373, 1247, 41268, 66508, 31]: continue
        # num_runs += 1
    

        # print ("offline_logfile",offline_logfile)
        
#         display (run.summary['test/test_fwd_pct_correct'] )#["filter_samples"].iloc[0])
        max_bwd = run.history()["test/test_bwd_pct_correct"].max()
        max_fwd = run.history()["test/test_fwd_pct_correct"].max()
        # max_bwd = run.history().iloc[14]["test/test_bwd_pct_correct"]
        # max_fwd = run.history().iloc[14]["test/test_fwd_pct_correct"]
        test_bwd_pct_correct.append(max_bwd)
        test_fwd_pct_correct.append(max_fwd)
        test_both_pct_correct.append((max_bwd+max_fwd)/2)
        num_sample_list.append(num_samples)
        run_type_list.append(run_type)
    
        # try:
        #     test_bwd_pct_correct.append( run.history().iloc[19]["test/test_bwd_pct_correct"])
        #     test_fwd_pct_correct.append( run.history().iloc[19]["test/test_fwd_pct_correct"])
        # except:
        #     continue
    return test_bwd_pct_correct, test_fwd_pct_correct, test_both_pct_correct, num_sample_list, run_type_list
dfs = []
group_name_lists = ["jan22_2221_pruning_500", "jan22_2221_pruning_1000", 
                    "jan22_2221_pruning_1500_from_8000", "jan22_2221_pruning_1500_from_9000",
                    "jan22_2221_pruning_2000_from_7500", "jan22_2221_pruning_2000_from_9000",
                    "jan22_2221_pruning_2500_from_7000", "jan22_2221_pruning_2500_from_9000",
                    "jan22_2221_pruning_3000_from_6500", "jan22_2221_pruning_3000_from_9000",
                    "jan22_2221_pruning_3500_from_6000", "jan22_2221_pruning_3500_from_9000",
                      "jan22_2221_pruning_4000_from_5500", "jan22_2221_pruning_4000_from_9000",
                    "jan22_2221_pruning_4500_from_5000", "jan22_2221_pruning_4500_from_9000",
                    "jan22_2221_pruning_5000_from_4500", "jan22_2221_pruning_5000_from_9000",
                    "jan22_2221_pruning_5500_from_4000", "jan22_2221_pruning_5500_from_9000",
                    "jan22_2221_pruning_6000_from_3500", "jan22_2221_pruning_6000_from_9000",
                                                         "jan22_2221_pruning_6500_from_9000",
                                                         "jan22_2221_pruning_7000_from_9000",
                                                         "jan22_2221_pruning_7500_from_9000",
                                                         "jan22_2221_pruning_8000_from_9000",
                                                         "jan22_2221_pruning_8500_from_9000",
                    "forwardprune_jan22_2221_pruning_6000_from_6500",
                    "forwardprune_jan22_2221_pruning_6500_from_7000",
                    "forwardprune_jan22_2221_pruning_7000_from_7500",
                    "forwardprune_jan22_2221_pruning_7500_from_8000",
                    "forwardprune_jan22_2221_pruning_8000_from_8500",
                   ]
num_samples_lists = [8500, 8000,
                    7500, 7500,
                    7000, 7000,
                    6500, 6500,
                    6000, 6000,
                    5500, 5500,
                    5000, 5000,
                    4500, 4500,
                    4000, 4000,
                    3500, 3500,
                    3000, 3000,
                          2500,
                          2000,
                          1500,
                          1000,
                          500,
                     3000,
                     2500,
                     2000,
                     1500,
                     1000
                    ]
run_type_lists = ["random", "random",
                  "backward", "random",
                  "backward", "random",
                  "backward", "random",
                  "backward", "random",
                  "backward", "random",
                  "backward", "random",
                  "backward", "random",
                  "backward", "random",
                  "backward", "random",
                  "backward", "random",
                              "random",
                              "random",
                              "random",
                              "random",
                              "random",
                  "forward",
                  "forward",
                  "forward",
                  "forward",
                  "forward"
                 ]
for (group_name, num_samples, run_type) in zip(group_name_lists, num_samples_lists, run_type_lists):
    test_bwd_pct_correct, test_fwd_pct_correct, test_both_pct_correct, num_sample_list, run_type_list = load_runs(group_name, num_samples, run_type)
    df = pd.DataFrame({"test_bwd_pct_correct": test_bwd_pct_correct, "test_fwd_pct_correct": test_fwd_pct_correct, "test_both_pct_correct": test_both_pct_correct, "num_samples": num_sample_list,
                    "run_type_list": run_type_list})
    dfs.append(df)

dfs = pd.concat(dfs)
x

14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
0
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
4
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
5
14
14
14
5
14
14
5
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
3
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
4
14
14
4
4
14
14
14
14
14
14
4
14
14
14
4
14
4
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
1

14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 1346

14
14765  8367 59586 65031 26745 44879 49577 47943  6332  8286 65352 34137
 52228 20496 46281 30415 25
14
14940 22900 44173  4691 18900 54907 65008 21325 41082 59975 27813 45413
 69177 55847 61540 50289 39
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 1346

14
25725 47509 62949 64208 61082  3715 35394  6938 16532 21609 45936  3755
 63034 68699 72122  2999 34
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 1346

14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
14940 22900 44173  4691 18900 54907 65008 21325 41082 59975 27813 45413
 69177 55847 61540 50289 39
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 1346

...

14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 1346

14
53037 12386 47700 38236 13849 15573 15977  7651 42445 66866 58171 60104
 64850 46443 13102 60858 53
14
32071 68166  8426 62749 30925 22923 23984 60159 57814 49616 24304 12725
  7699 37769 27901 33585 42
14
 5696 22233 41203 64169 50644  1273 31503 71701  6657  5830 51570 15510
 13682 39177 31714  6752 28
14
47128  5165   992 64677 23595 72710 27131 38338 28473 18736 62393 24748
 25322  7264 26277 28895 41
14
70074 18334 61414  3301 38906 29815  8862 26735 47561 10632  3811  9255
 45140 38663 19653 21570  4
14
12466 35275 21945 70706 26861  4729 23527 43968 15143 23865  9712 23401
 69318 20562 39745 51251 33
14
68355 12593 28925  5558 62741 53540  3182 30013 26070 31262 65966 22236
 57953 24390 61875 28454 68
14
57994 13134  4343 16735 50200 29881 42538  3276 44378 36834 28084 63852
 57550  6876 72645  5918 36
14
52484 15541 26504 70327 38634 45699  5121 65387 31905 41540  1396 56225
 57119 35354 27685 40061  6
14
21054 13986 51830 10255 36814 34826 37920 56873 66520 21520 51215 1046

14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 1346

14
57994 13134  4343 16735 50200 29881 42538  3276 44378 36834 28084 63852
 57550  6876 72645  5918 36
14
21937 41377  3925 38670  8983 52767 54049 22381 19384 65778 61171 61344
 46153 70960 72095  2264 69
14
68355 12593 28925  5558 62741 53540  3182 30013 26070 31262 65966 22236
 57953 24390 61875 28454 68
14
65993 65339 20951 42052 68166  8389 55638 17636 16727 23739  6044 29492
 10355 36841 49375 22120  6
14
29833 11018 37621 42945 58279 55878 66512 27976 57119 11655 70569 46683
 60302 51697 30748 29349 15
14
52484 15541 26504 70327 38634 45699  5121 65387 31905 41540  1396 56225
 57119 35354 27685 40061  6
14
55172 51913  5756 37680 42052 43812 20013 32931 67428 50259 28929 47042
 49748 28003 35832 70833 58
14
57729 26478 18648 14135 35176 71330 14920 32464 56951 52971 12229 19989
 59093 37998  4711  3893 15
14
55641 58046 33232 16962  1521  2962  8304 22237 61589 13289 25960 10459
 28525 20902 62054 34457 14
14
40881 68391 61461 10445 11551 24845 51830 40977 72226  4362 33866  823

14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
1

In [ ]:
pd.set_option('display.max_rows', 500)
# display(dfs)
sns.set_theme(style="whitegrid")
ax = sns.lineplot( x="num_samples", y="test_both_pct_correct", marker="o", hue="run_type_list", data=dfs,errorbar=lambda x:  (np.min(x),np.max(x)))
plt.show()

In [ ]:
dfs.to_csv("result_of_active_learning.csv")

# Main

In [ ]:
import webdataset as wds 
import random
import torch
import h5py
import numpy as np
import matplotlib.pyplot as plt
from qpsolvers import solve_qp

def my_split_by_node(urls): return urls

f = h5py.File(f'/weka/proj-fmri/shared/mindeyev2_dataset/betas_all_subj01_fp32.hdf5', 'r')
voxels = f['betas'][:]
del f 
print (voxels.shape)
train_url = "/weka/proj-fmri/shared/mindeyev2_dataset/wds/subj01/new_train/{0..39}.tar"
train_data = wds.WebDataset(train_url,resampled=True,nodesplitter=my_split_by_node)\
                        .shuffle(750, initial=2500, rng=random.Random(1))\
                        .decode("torch")\
                       .rename(behav="behav.npy", past_behav="past_behav.npy", future_behav="future_behav.npy", olds_behav="olds_behav.npy")\
                       .to_tuple(*["__key__", "behav", "past_behav", "future_behav", "olds_behav"]) 
train_dl = torch.utils.data.DataLoader(train_data, batch_size=512, shuffle=False, drop_last=True, pin_memory=True) 
keys = set() 
for iter, (key, behav0, past_behav0, future_behav0, old_behav0) in enumerate(train_dl): 
    
#     print (behav0[:,0,0].cpu().long())
#     print (past_behav0[:,0,0].cpu().long())
    [keys.add( i.item() ) for i in behav0[:,0,5].cpu().long()]
#     print ((keys))
    if iter % 10 == 1: print (len(keys))
    if len(keys) >= 27000: break

(30000, 15724)
1024
5494
9393
11333
12820
14439
15528
17256
18204
20076
22666
23228
24325
25008
25735
26331
26333
26333
26333
26333
26333
26816


In [ ]:
# get voxels using keys
keys = list(keys)
_voxels = voxels[keys]
# convert to numpy
print (_voxels.shape)

(27000, 15724)


In [3]:
from sklearn.cluster import KMeans
n_clusters = 100
kmeans = KMeans(n_clusters=n_clusters, random_state=1, n_init="auto",verbose=2).fit(_voxels)
kmeans.labels_

Initialization complete
Iteration 0, inertia 583358912.0.
Iteration 1, inertia 358514368.0.
Iteration 2, inertia 354550368.0.
Iteration 3, inertia 353339264.0.
Iteration 4, inertia 352748896.0.
Iteration 5, inertia 352369568.0.
Iteration 6, inertia 352097696.0.
Iteration 7, inertia 351907776.0.
Iteration 8, inertia 351776256.0.
Iteration 9, inertia 351673472.0.
Iteration 10, inertia 351594560.0.
Iteration 11, inertia 351537088.0.
Iteration 12, inertia 351491232.0.
Iteration 13, inertia 351456160.0.
Iteration 14, inertia 351421376.0.
Iteration 15, inertia 351392192.0.
Iteration 16, inertia 351361984.0.
Iteration 17, inertia 351335808.0.
Iteration 18, inertia 351311936.0.
Iteration 19, inertia 351289184.0.
Iteration 20, inertia 351270528.0.
Iteration 21, inertia 351254976.0.
Iteration 22, inertia 351240192.0.
Iteration 23, inertia 351228032.0.
Iteration 24, inertia 351217472.0.
Iteration 25, inertia 351206208.0.
Iteration 26, inertia 351196288.0.
Iteration 27, inertia 351187648.0.
Iterat

array([71, 30, 85, ..., 82, 98, 44], dtype=int32)

In [4]:
# Calculate dintra and dinter for each cluster. 
# dintra is computed as the average cosine distance between the items of a cluster and its centroid
# dinter is computed for each cluster j as the average cosine distance between a cluster centroid and its l nearest
# neighbor centroids
def compute_dintra_dinter(kmeans, n_neighbors = 20, n_clusters = 100): 
    """
    kmeans: kmeans object  
    n_neighbors: number of neighbors to use for computing dinter
    n_clusters: number of clusters
    """
    d_intra = np.zeros(n_clusters)
    d_inter = np.zeros(n_clusters)
    
    for cluster_id in range(n_clusters):
        cluster = _voxels[kmeans.labels_ == cluster_id]
        # d_intra[cluster_id] = (np.linalg.norm(cluster - kmeans.cluster_centers_[cluster_id], axis=1).mean())
        a_dot_b = np.dot(cluster, np.expand_dims(kmeans.cluster_centers_[cluster_id], axis=1))
        a_norm = np.linalg.norm(cluster, axis=1) 
        b_norm = np.linalg.norm(kmeans.cluster_centers_[cluster_id])
        # print (a_dot_b.shape, a_norm.shape, b_norm, cluster.shape, np.expand_dims(kmeans.cluster_centers_[cluster_id], axis=1).shape)
        d_intra[cluster_id] = 1-(a_dot_b.squeeze(1) / (a_norm * b_norm)).mean()

        # compute distance from this cluster to all other clusters
        a_dot_b = np.dot(np.expand_dims(kmeans.cluster_centers_[cluster_id], axis=0), kmeans.cluster_centers_.T)
        a_norm = np.linalg.norm(np.expand_dims(kmeans.cluster_centers_[cluster_id], axis=0), axis=1)
        b_norm = np.linalg.norm(kmeans.cluster_centers_, axis=1)
        # print (kmeans.cluster_centers_[cluster_id].shape, kmeans.cluster_centers_.T.shape, a_dot_b.shape, a_norm.shape, b_norm.shape)
        distance_to_other_clusters = 1-(a_dot_b / (a_norm * b_norm))  
        d_inter[cluster_id] = np.sort(distance_to_other_clusters)[0,1:(n_neighbors+1)].mean() # 0 is the distance to itself

    return d_intra, d_inter

def pruning(kmeans, d_intra, d_inter, n_clusters = 100, tau = 0.1, fraction = 0.5, N = 27000): 
    """
    kmeans: kmeans object  
    d_intra: dintra for each cluster
    d_inter: dinter for each cluster
    n_clusters: number of clusters
    tau: temperature for softmax
    fraction: fraction of the dataset to use for computing Nj
    N: target dataset size
    """
    # print ("distance_to_other_clusters", distance_to_other_clusters, "distance_closest_neighbors", distance_closest_neighbors) 
    # Calculate the number of examples per cluster Nj. 
    Cj = d_intra * d_inter
    # softmax
    Pj = np.exp(Cj/tau) / np.exp(Cj/tau).sum()
    Nj = Pj * N 

    # solve quadratic program to fix Nj
    probs = torch.softmax(torch.tensor(Cj/tau),dim=0)
    # print(probs,probs.shape,Pj)
    P = np.eye(n_clusters)
    q = - Nj
    A = np.array([1.0] * n_clusters)
    b = np.array([N])
    # Define the lower and upper bounds
    min_samples = 1
    num_items_in_each_cluster = np.bincount(kmeans.labels_)
    bounds = np.array([ ( min_samples, num_items_in_each_cluster[i] ) for i in range(n_clusters) ])
    # print (bounds.shape,bounds[:,[0]].shape,bounds[:,[1]].shape,P.shape, q.shape)
    X = solve_qp(P=P, q=q, A=A, b=b, lb=bounds[:,0].reshape(1,n_clusters), ub=bounds[:,1].reshape(1,n_clusters), solver="osqp")
    Nj = np.rint(X).astype(int)
    print(Nj)
    # print(Nj)
    pruned_ids = []
    for cluster_id in range(n_clusters):
        
        cluster_ids = np.where( kmeans.labels_ == cluster_id)[0]
        
        cluster = _voxels[cluster_ids]
        nsamples_cluster = Nj[cluster_id]
        # compute cosine similarity of cluster to cluster_centers_[cluster_id]
        a_dot_b = np.dot(cluster, np.expand_dims(kmeans.cluster_centers_[cluster_id], axis=1))
        a_norm = np.linalg.norm(cluster, axis=1)
        b_norm = np.linalg.norm(kmeans.cluster_centers_[cluster_id])
        distance_to_centroid = 1-(a_dot_b.squeeze(1) / (a_norm * b_norm))
        distance_to_centroid = np.argsort(distance_to_centroid)[-nsamples_cluster:]
        pruned_ids.extend(cluster_ids[distance_to_centroid])
        # select nsamples_cluster from cluster that are furthest away from cluster centroid
        # print (cluster.shape, distance_to_centroid.shape)

    return pruned_ids, Nj, Pj
d_intra, d_inter = compute_dintra_dinter(kmeans)
seed=1
for N in [750, 1500, 2250, 3000, 6000, 9000, 12000, 15000, 18000, 21000, 24000, 27000]:
    pruned_ids, Nj, Pj = pruning(kmeans, d_intra, d_inter, N=N)
    np.save(f"./cache/kmeans_pruned_ids_N_{int(N)}_seed_{seed}.npy", pruned_ids)
    print ("N", N, pruned_ids[:100], len(pruned_ids))
    # plt.scatter(d_intra,d_inter)
    # plt.show()
    # plt.plot(Pj) 



/weka/home-alexnguyen/mind-reader/MindEyeV2/src/mindeye/lib/python3.11/site-packages/qpsolvers/conversions/ensure_sparse_matrices.py:24: UserWarning: Converted P to scipy.sparse.csc.csc_matrix
For best performance, build P as a scipy.sparse.csc_matrix rather than as a numpy.ndarray
  warnings.warn(
/weka/home-alexnguyen/mind-reader/MindEyeV2/src/mindeye/lib/python3.11/site-packages/qpsolvers/conversions/ensure_sparse_matrices.py:24: UserWarning: Converted A to scipy.sparse.csc.csc_matrix
For best performance, build A as a scipy.sparse.csc_matrix rather than as a numpy.ndarray
  warnings.warn(


[ 3  6 31  4  4  6  3 10 49 16 30 14  3  2  9  1  1 14 10  4  1 13 15  1
  1  1  4 16 18  4 28  7  1  1 22  9 11 12  2  7  1  4 13  4  7  3  1  5
  8  3 10  1  4  8  2  1  1  1  2 20  1  4  2  1  1  3  4  1 18  1  1  1
  1  2  8  8  7  3  1  8 17  5 63  1  4  2  8 12 20  6  1  3  6  2 14  7
  3  1  6  2]
N 750 [22709, 26150, 9107, 8155, 8156, 8166, 8165, 8167, 8173, 12540, 14807, 12857, 25158, 21029, 19557, 3324, 20313, 6433, 22757, 10722, 2016, 23515, 15448, 20699, 13775, 21678, 16268, 12589, 18879, 6637, 8408, 25012, 8060, 10220, 23642, 803, 19434, 19414, 14624, 20698, 733, 23981, 2658, 19217, 23059, 23012, 23014, 23116, 24157, 24134, 24162, 24219, 24269, 7905, 11655, 7014, 13608, 7600, 7605, 7586, 7584, 7581, 7590, 7601, 7602, 7604, 7580, 15421, 11104, 16868, 5595, 7753, 19447, 6455, 15770, 7336, 15457, 26435, 13632, 16586, 14130, 15481, 6187, 11480, 4998, 17975, 8702, 6790, 2164, 8533, 7909, 7894, 23950, 16652, 15887, 9476, 26582, 15943, 5806, 21561] 752
[  7  13  62   7   9  13   

In [27]:
np.unique(pruned_ids).shape

(14228,)

In [70]:

P = np.eye(n_clusters)
q = - Nj
A = np.array(1.0 * n_clusters)
b = np.array([N])
# Define the lower and upper bounds
min_samples = 1
num_items_in_each_cluster = np.bincount(kmeans.labels_)
bounds = np.array([ ( min_samples, num_items_in_each_cluster[i] ) for i in range(n_clusters) ])
X = solve_qp(P=P, q=q, A=A, b=b, lb=bounds[:,0], ub=bounds[:,1], solver="osqp")
X = np.rint(X).astype(int)

SolverNotFound: found solvers ['clarabel'] but 'osqp' is not one of them; if 'osqp' is listed in https://github.com/qpsolvers/qpsolvers#solvers you can run ``pip install qpsolvers[osqp]``

In [ ]:
plt.plot(losses)
plt.show()
plt.plot(test_losses)
plt.show()